In [1]:
using Random
import SymbolicRegression: Options, equation_search
using SymbolicRegression
using SpecialFunctions
using Dates
using Statistics
using Printf

In [32]:
# Generate the X and y data

const alpha = 100.0f0
const weight = 0.5f0

X_vec = collect(LinRange(95, 300, 500))
y_vec = besselj.(alpha, X_vec)

# Reshape to Julia's standard (features, samples) format
X = reshape(X_vec, 1, length(X_vec))
# y should be a vector
y = y_vec

X_asymptotic = LinRange(1000.0f0, 1500.0f0, 100)

dataset = Dataset(X, y)

println("Data generated successfully.")
println("Shape of X: ", size(X))
println("Shape of y: ", size(y))
println("Shape of X_asymptotic: ", size(X_asymptotic))

In [ ]:
# 1. Set up the output directory and file path
timestamp = Dates.format(now(), "yyyymmdd_HHMMSS")
output_dir = joinpath("pysr_runs", "Julia_run_$(timestamp)")
mkpath(output_dir)

println("Results will be saved to directory: ", abspath(output_dir))

# 2. Define the template using the macro
expression_spec = @template_spec(expressions=(A, phi)) do X1
    A(X1) * cos(phi(X1))
end

# 3. Set the options for the search
hall_of_fame_options = Options(
    binary_operators=[+, *, -, /],
    unary_operators=[sqrt],
    maxsize=60,
    output_directory=output_dir,
    # --- The template specification MUST go here, inside Options ---
    expression_spec=expression_spec
)

# 4. Run the equation search
hall_of_fame = EquationSearch(
    X, y;
    niterations=500,
    options=hall_of_fame_options,
    parallelism=:multithreading,
    # --- The variable names MUST go here, in EquationSearch ---
    variable_names=["X1"]
)

println("\nSearch complete using the specified template. Results saved.")

In [8]:
# ===================================================================
# SCRIPT WITH DATA FIT + ASYMPTOTIC LOSS
# ===================================================================

using SymbolicRegression
using SpecialFunctions
using Statistics
using Dates
using Printf
using DynamicExpressions

# --- 1. Hyperparameters and Data Setup ---
const alpha = 100.0f0
const weight_asymptotic = 0.0001f0 # Re-introducing the weight for this loss part

# --- Fitting Data ---
X_vec = collect(LinRange(95.0f0, 300.0f0, 500))
y_vec = besselj.(alpha, X_vec)
X = reshape(X_vec, 1, length(X_vec))
y = y_vec

# --- Asymptotic Data ---
X_asymptotic_vec = collect(LinRange(1000.0f0, 1500.0f0, 100))
X_asymptotic = reshape(X_asymptotic_vec, 1, length(X_asymptotic_vec))

# --- Create the Dataset object ---
dataset = Dataset(X, y; variable_names=["x"])
println("Data prepared successfully.")


# --- 2. Loss Function with Asymptotic Term ---
function data_plus_asymptotic_loss(tree, dataset, options)::Float32
    # PART A: Data Fit Loss (we know this works)
    y_pred, completed = eval_tree_array(tree, dataset.X, options)
    !completed && return Inf32
    data_mse = mean((y_pred .- dataset.y).^2)

    # PART B: Asymptotic Violation Loss
    tree_A = tree.trees.A
    tree_phi = tree.trees.phi
    
    A_asymptotic, completed_A = eval_tree_array(tree_A.tree, X_asymptotic, options)
    phi_asymptotic, completed_phi = eval_tree_array(tree_phi.tree, X_asymptotic, options)
    
    if !completed_A || !completed_phi
        return Inf32
    end
    
    A_target_asymptotic = @. sqrt(2.0f0 / (π * X_asymptotic))
    phi_target_asymptotic = @. X_asymptotic - (π/2 * alpha) - (π/4)
    
    ratio_A = A_asymptotic ./ A_target_asymptotic
    
    asymptotic_mse_A = mean((ratio_A .- 1.0f0).^2)
    asymptotic_mse_phi = mean((phi_asymptotic .- phi_target_asymptotic).^2)
    total_asymptotic_loss = asymptotic_mse_A + asymptotic_mse_phi

    # PART C: Return the combined, weighted loss
    return data_mse + weight_asymptotic * total_asymptotic_loss
end


# ===================================================================
# 3. Main Search Execution
# ===================================================================

timestamp = Dates.format(now(), "yyyymmdd_HHMMSS")
output_dir = joinpath("pysr_runs", "Julia_run_$(timestamp)")
mkpath(output_dir)
println("Results will be saved to directory: ", abspath(output_dir))

expression_spec = @template_spec(expressions=(A, phi)) do x
    A(x) * cos(phi(x))
end

options = Options(
    binary_operators=[+, *, -, /],
    unary_operators=[sqrt],
    maxsize=40,
    loss_function_expression=data_plus_asymptotic_loss, # Using the new version
    output_directory=output_dir,
    expression_spec=expression_spec
)

println("Starting equation search with Data + Asymptotic loss...")
hall_of_fame = EquationSearch(
    dataset;
    niterations=70, # Increased iterations for the more complex task
    options=options,
    parallelism=:multithreading
)

println("\nSearch complete.")